In [1]:
import quippy
import numpy as np
import os
import matplotlib as mpl
import matplotlib.pyplot as plt
import StringIO
import glob
import sys
import ase

In [4]:
this_dir = os.getcwd()

In [7]:

root_mbx_functions=(this_dir.split("tutorials")[0])
sys.path.insert(0,root_mbx_functions)
import mbx_functions
import unwrap_oh2_ch4

In [8]:
from CalcMBXnonperiodic import CalcMBX
potMBX_non_periodic = CalcMBX()

In [9]:
from SuperCellCalcMBX import SuperCellCalcMBX
potMBX_periodic = SuperCellCalcMBX()

In [10]:
SuperCellCalcMBX?

```Init signature: SuperCellCalcMBX(self, min_box_requirement=24.0, my_json='mbx_pbc.json', **kwargs)
```

one can specify to use another json file in which different polynomials can be turned off (to use the classical force field's terms instead)



an example json file:
```{
   "Note" : "This is a cofiguration file",
   "MBX" : {
       "box" : [50.0,0.0,0.0,0.0,50.0,0.0,0.0,0.0,50.0],
       "twobody_cutoff"   : 9.0,
       "threebody_cutoff" : 7.0,
       "max_n_eval_1b"    : 500,
       "max_n_eval_2b"    : 500,
       "max_n_eval_3b"    : 500,
       "dipole_tolerance" : 1E-16,
       "dipole_max_it"    : 100,
       "dipole_method"     : "cg",
       "alpha_ewald_elec" : 0.6,
       "grid_density_elec" : 2.5,
       "spline_order_elec" : 6,
       "alpha_ewald_disp" : 0.6,
       "grid_density_disp" : 2.5,
       "spline_order_disp" : 6,
       "ttm_pairs" : [],
       "ignore_2b_poly" : [],
       "ignore_3b_poly" : []
   } ,
   "i-pi" : {
       "port" : 34543,
       "localhost" : "localhost"
   }
}
```

If modifying the json files, one can leave most of the parameters as they are (the lattice will be set to the structures lattice in QUIP).

Certain PIPs can be turned off by modifying the following lines:
```
       "ignore_2b_poly" : [],
       "ignore_3b_poly" : []
```
to e.g.:
```
       "ignore_2b_poly" : [],
       "ignore_3b_poly" : [["ch4","h2o","h2o"]]
```
in case we would like to test the version without the methane-water-water 3B fit.

### Calculations with the above defined potentials:

#### Non-periodic calls:

In [11]:
## read in the Atoms object:
at_m1w5 = ase.io.read("m1w5_fixed.xyz")

In [12]:
at_m1w5

Atoms(symbols='CH4OH2OH2OH2OH2OH2', pbc=False)

In [13]:
## assign the calculator to the Atoms object:
at_m1w5.set_calculator(potMBX_non_periodic)

In [14]:
## perform energy calculation:
e = at_m1w5.get_potential_energy()
print e

-0.4057065229


#### Periodic calls

In [16]:
## read in the Atoms object:
at_clathrate = ase.io.read("phase_clathrate.xyz")

In [17]:
at_clathrate

Atoms(symbols='C8H124O46', pbc=True, cell=[11.7429, 11.7429, 11.7429], map_shift=..., n_neighb=...)

In [18]:
## assign the calculator to the Atoms object:
at_clathrate.set_calculator(potMBX_periodic)

In [19]:
## perform energy calculation:
e = at_clathrate.get_potential_energy()
print e

-29.0782165281


### one can look at the atoms object in ase:

In [20]:
from ase.visualize import view

In [21]:
view(at_m1w5)

In [22]:
view(at_clathrate)

### "Unwrapping" structures
The ase Calculator object takes care of "unwrapping" the structure
but in case we'd like to use MBX without the ase helper functions,
we need an input where the molecules are not cut into halves at the pbc.

In [23]:
at_clathrate_unwrapped_dict = unwrap_oh2_ch4.unwrap_water_methane(at_clathrate)

In [24]:
at_clathrate_unwrapped_dict

{'phase_m': Atoms(symbols='CH4CH4CH4CH4CH4CH4CH4CH4', pbc=True, cell=[11.7429, 11.7429, 11.7429], map_shift=..., n_neighb=...),
 'phase_mw': Atoms(symbols='C8H124O46', pbc=True, cell=[11.7429, 11.7429, 11.7429], map_shift=..., n_neighb=...),
 'phase_w': Atoms(symbols='H92O46', pbc=True, cell=[11.7429, 11.7429, 11.7429], map_shift=..., n_neighb=...)}

In [25]:
at_clathrate_unwrapped = at_clathrate_unwrapped_dict["phase_mw"]

In [26]:
view(at_clathrate_unwrapped)

In [30]:
ase.io.write("phase_clathrate_unwrapped.xyz", at_clathrate_unwrapped)

### Building supercells
MBX needs the Atoms object to have at least 24. A boxlengths.
The ase Calculator also builds the supercells of the Atoms object but if needed, we can create an input as:

#### Don't forget to firstly unwrap the cell!
(The supercell would work but MBX needs the atoms to be ordered according to the molecules)

In [32]:
at_clathrate_unwrapped.cell

array([[11.7429,  0.    ,  0.    ],
       [ 0.    , 11.7429,  0.    ],
       [ 0.    ,  0.    , 11.7429]])

In [42]:
min_box_length=24.

In [58]:
lattice = at_clathrate_unwrapped.cell
s_i_list =[1,1,1]
for i in range(3):
    lat_ii=lattice[i][i]
    s_i_list[i]= (int(np.ceil(min_box_length/lat_ii)))
print s_i_list

[3, 3, 3]


In [59]:
at_super = at_clathrate_unwrapped.copy()*(s_i_list)

In [60]:
len(at_super)/len(at_clathrate_unwrapped)

27

In [61]:
ase.io.write("phase_clathrate_unwrapped_supercell.xyz", at_super)